# 03 – Pohrana u SQLite bazu (pzap.db)

U ovom koraku spremamo integrirane podatke u SQLite bazu radi lakšeg upita i korištenja u REST API-ju.

Tablice:
- **dt_yearly** (DT: godišnje + daily_minutes)
- **wb_internet_year** (World Bank: internet_pct po godini)
- **integrated_individual** (SMP + agregati DT/WB + TW agregati)


Putanje + učitavanje “finalnog” CSV-a

In [ ]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parent
DATA_PROCESSED = PROJECT_ROOT / "data_processed"
DATA_RAW = PROJECT_ROOT / "data_raw"

INTEGRATED_CSV = DATA_PROCESSED / "integrated_individual_level.csv"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("INTEGRATED_CSV exists:", INTEGRATED_CSV.exists())

integrated_individual = pd.read_csv(INTEGRATED_CSV)
display(integrated_individual.head())
print("integrated_individual shape:", integrated_individual.shape)


PROJECT_ROOT: d:\Preuzimanja\PZAP_PROJEKT\PAP_PROJEKT
INTEGRATED_CSV exists: True


,age,gender,job_type,daily_social_media_time,number_of_notifications,work_hours_per_day,perceived_productivity_score,actual_productivity_score,stress_level,social_platform_preference,dt_daily_minutes_avg,internet_pct_avg_dt_years,tw_total_time_spent_mean,tw_sessions_mean,tw_engagement_mean
0,56,Male,Unemployed,4.180940,61,6.753558,8.040464,7.291555,4.0,Facebook,129.230769,56.200389,151.406,10.013,4997.159
1,46,Male,Health,3.249603,59,9.169296,5.063368,5.165093,7.0,Twitter,129.230769,56.200389,151.406,10.013,4997.159
2,56,Female,Unemployed,4.381070,60,3.902309,6.420989,5.976408,7.0,TikTok,129.230769,56.200389,151.406,10.013,4997.159
3,36,Female,Education,4.089168,49,6.560467,2.681830,2.446927,4.0,Twitter,129.230769,56.200389,151.406,10.013,4997.159
4,40,Female,Education,4.097401,57,5.839590,3.219022,3.004240,4.0,Instagram,129.230769,56.200389,151.406,10.013,4997.159


integrated_individual shape: (23730, 15)
